In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

from sklearn import linear_model
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [45]:
def point_vizual(point):#на вход принимает словaрь с команда:очки
    point_df = pd.DataFrame({
        'счет': list(point.values())
    })
    point_df.index = list(d.keys())
    point_df.sort_values(by="счет",axis=0,ascending=False)
    return(point_df)

def point_bund(df):
    n = list(df["Команда1"].value_counts().index)
    d = dict()
    for i in n:
        d.update({i:0})
    df = df.sort_values(by="Tour",axis=0)
    for i in list(df["id"]):
        num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
        
        df.loc[df["id"] == i ,["Point_1"]] = d[num[0]]
        df.loc[df["id"] == i ,["Point_2"]]  = d[num[2]]
        
        if num[1] > num[3]:
            d[num[0]] +=  3
        elif num[1] < num[3]:
            d[num[2]] +=  3
        else:
            d[num[0]] +=  1
            d[num[2]] +=  1
    return df

def preobr(df):
    if 'Unnamed: 0.1' in list(df.columns):
        df = df.drop('Unnamed: 0.1',axis=1)
    if 'Unnamed: 0' in list(df.columns):
        df = df.drop('Unnamed: 0',axis=1)
    
    df = df.drop('Матчи 1',axis=1)
    df = df.drop('Матчи 2',axis=1)
    df = df.astype({'счет1': 'int32'})
    df = df.astype({'счет2': 'int32'})

    df["тотал"] = df["счет1"] + df["счет2"]
    df["Дата"] = pd.to_datetime(df['Дата'],dayfirst=True)
    df['Температура'] = df['Температура'].apply(lambda x: int(x[1:-2]))
    
    for i in [14919463, 14898101]:
        df.loc[df["id"] == i ,['Офсайды  (соперник) 2']] = 4.00
        df.loc[df["id"] == i ,['Офсайды 2']] = 1.00
    
    #mask_1 = df["id"] == 14898101
    #df['Офсайды  (соперник) 2'][mask_1] = 4.00
    #df['Офсайды 2'][mask_1] = 1.00
        
    df["Point_1"] = 0
    df["Point_2"] = 0
    df["liga"] = df["общая информация"].apply(lambda x: x.split(",")[0])
    df["Tour"] = df["общая информация"].apply(lambda x: x.split(",")[1])
    
    df = df[df["Tour"] != " стыковые матчи"]
    df = df[df["Tour"] != " понижение/повышение - финал"]
    df = df[df["Tour"] != " дополнительный матч"]
    df["Tour"] = df["Tour"].apply(lambda x: int(x.strip()[:-6]))
    
    df = df.astype({'Офсайды  (соперник) 2': 'float32'})
    df = df.astype({'Офсайды 2': 'float32'})

    df["Сухие матчи 1"] = df["Сухие матчи 1"].apply(lambda x: int(x.split()[-1]))
    df["Сухие матчи 2"] = df["Сухие матчи 2"].apply(lambda x: int(x.split()[0]))

    df["обе забьют 1"] = df["обе забьют 1"].apply(lambda x: int(x.split()[-1]))
    df["обе забьют 2"] = df["обе забьют 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал больше 2.5 1"] = df["Тотал больше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал больше 2.5 2"] = df["Тотал больше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал меньше 2.5 1"] = df["Тотал меньше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал меньше 2.5 2"] = df["Тотал меньше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Поражения 1"] = df["Поражения 1"].apply(lambda x: int(x.split()[-1]))
    df["Поражения 2"] = df["Поражения 2"].apply(lambda x: int(x.split()[0]))

    df["Ничьи 1"] = df["Ничьи 1"].apply(lambda x: int(x.split()[-1]))
    df["Ничьи 2"] = df["Ничьи 2"].apply(lambda x: int(x.split()[0]))

    df["Победы 1"] = df["Победы 1"].apply(lambda x: int(x.split()[-1]))
    df["Победы 2"] = df["Победы 2"].apply(lambda x: int(x.split()[0]))

    df["Отдых: дни (часы) 1"] = df["Отдых: дни (часы) 1"].apply(lambda x: float(x.split()[-1]))
    df["Отдых: дни (часы) 2"] = df["Отдых: дни (часы) 2"].apply(lambda x: float(x.split()[0]))
    
    df = point_bund(df)
    return df

In [46]:
Bundes_liga_19_20 = pd.read_csv("data_auto/Bundes_lige2019-2020.csv")
Bundes_liga_20_21 = pd.read_csv("data_auto/Bundes_lige2020-2021.csv")
Bundes_liga_21_22 = pd.read_csv("data_auto/Bundes_lige2021-2022.csv")
Bundes_liga_22_23 = pd.read_csv("data_auto/Bundes_lige2022-2023.csv")

Premier_liga19_20 = pd.read_csv("data_auto/Premier_liga2019-2020.csv")
Premier_liga20_21 = pd.read_csv("data_auto/Premier_liga2020-2021.csv")
Premier_liga21_22 = pd.read_csv("data_auto/Premier_liga2021-2022.csv")
Premier_liga22_23 = pd.read_csv("data_auto/Premier_liga2022-2023.csv")

Seria_A_liga19_20 = pd.read_csv("data_auto/Seria_A_liga2019-2020.csv")
Seria_A_liga20_21 = pd.read_csv("data_auto/Seria_A_liga2020-2021.csv")
Seria_A_liga21_22 = pd.read_csv("data_auto/Seria_A_liga2021-2022.csv")
Seria_A_liga22_23 = pd.read_csv("data_auto/Seria_A_liga2022-2023.csv")

Primera_liga19_20 = pd.read_csv("data_auto/Primera_liga2019-2020.csv")
Primera_liga20_21 = pd.read_csv("data_auto/Primera_liga2020-2021.csv")
Primera_liga21_22 = pd.read_csv("data_auto/Primera_liga2021-2022.csv")
Primera_liga22_23 = pd.read_csv("data_auto/Primera_liga2022-2023.csv")

In [47]:
Bundes_liga_19_20_cl = preobr(Bundes_liga_19_20)
Bundes_liga_20_21_cl = preobr(Bundes_liga_20_21)
Bundes_liga_21_22_cl = preobr(Bundes_liga_21_22)
Bundes_liga_22_23_cl = preobr(Bundes_liga_22_23)

Premier_liga19_20_cl = preobr(Premier_liga19_20)
Premier_liga20_21_cl = preobr(Premier_liga20_21)
Premier_liga21_22_cl = preobr(Premier_liga21_22)
Premier_liga22_23_cl = preobr(Premier_liga22_23)

Seria_A_liga19_20_cl = preobr(Seria_A_liga19_20)
Seria_A_liga20_21_cl = preobr(Seria_A_liga20_21)
Seria_A_liga21_22_cl = preobr(Seria_A_liga21_22)
Seria_A_liga22_23_cl = preobr(Seria_A_liga22_23)

Primera_liga19_20_cl = preobr(Primera_liga19_20)
Primera_liga20_21_cl = preobr(Primera_liga20_21)
Primera_liga21_22_cl = preobr(Primera_liga21_22)
Primera_liga22_23_cl = preobr(Primera_liga22_23)

In [48]:
df = pd.concat([Bundes_liga_19_20_cl,
               Bundes_liga_20_21_cl,
               Bundes_liga_21_22_cl,
               Bundes_liga_22_23_cl,
               Premier_liga19_20_cl,
               Premier_liga20_21_cl,
               Premier_liga21_22_cl,
               Premier_liga22_23_cl,
               Seria_A_liga19_20_cl,
               Seria_A_liga20_21_cl,
               Seria_A_liga21_22_cl,
               Seria_A_liga22_23_cl,
               Primera_liga19_20_cl,
               Primera_liga20_21_cl,
               Primera_liga21_22_cl,
               Primera_liga22_23_cl])
df

,id,общая информация,Дата,Команда1,счет1,Команда2,счет2,Температура,Признак погоды,поб1,...,Желтые карточки (соперник) 2,Красные карточки 1,Красные карточки 2,Красные карточки (соперник) 1,Красные карточки (соперник) 2,тотал,Point_1,Point_2,liga,Tour
307,14898095,"Бундеслига, 1-й тур, регулярный сезон, 16.08.2...",2019-08-16 21:35:00,Бавария,2,Герта,2,20,переменная облачность,1.18,...,0.17,0.11,0.00,0.00,0.00,4,0,0,Бундеслига,1
299,14898103,"Бундеслига, 1-й тур, регулярный сезон, 18.08.2...",2019-08-18 19:00:00,Унион Берлин,0,РБ Лейпциг,4,19,переменная облачность,4.60,...,2.00,0.00,0.00,0.00,0.00,4,0,0,Бундеслига,1
300,14898100,"Бундеслига, 1-й тур, регулярный сезон, 18.08.2...",2019-08-18 16:30:00,Айнтрахт Франкфурт,1,Хоффенхайм,0,26,солнечно,2.11,...,2.25,0.00,0.13,0.00,0.00,1,0,0,Бундеслига,1
301,14898098,"Бундеслига, 1-й тур, регулярный сезон, 17.08.2...",2019-08-17 19:30:00,Боруссия М,0,Шальке,0,21,солнечно,2.25,...,1.00,0.00,0.00,0.14,0.00,0,0,0,Бундеслига,1
306,14898102,"Бундеслига, 1-й тур, регулярный сезон, 17.08.2...",2019-08-17 16:30:00,Фрайбург,3,Майнц,0,26,солнечно,2.69,...,2.00,0.20,0.00,0.00,0.14,3,0,0,Бундеслига,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,1744676,"Примера, 38-й тур, регулярный сезон, 04.06.202...",2023-06-04 22:00:00,Эспаньол,3,Альмерия,3,11,ясно,2.42,...,2.10,0.20,0.10,0.00,0.10,6,36,40,Примера,38
3,1744675,"Примера, 38-й тур, регулярный сезон, 04.06.202...",2023-06-04 22:00:00,Эльче,1,Кадис,1,15,ясно,2.51,...,2.80,0.10,0.00,0.20,0.40,2,24,41,Примера,38
2,1744674,"Примера, 38-й тур, регулярный сезон, 04.06.202...",2023-06-04 22:00:00,Реал Вальядолид,0,Хетафе,0,12,небольшой дождь,1.96,...,2.00,0.10,0.10,0.00,0.20,0,39,41,Примера,38
1,1744673,"Примера, 38-й тур, регулярный сезон, 04.06.202...",2023-06-04 22:00:00,Сельта,2,Барселона,1,12,переменная облачность,2.81,...,2.00,0.20,0.00,0.30,0.30,3,40,88,Примера,38


In [51]:
def point_bund(df):
    n = list(df["Команда1"].value_counts().index)
    d = dict()
    for i in n:
        d.update({i:0})
    df = df.sort_values(by="Tour",axis=0)
    for i in list(df["id"]):
        num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
        
        df.loc[df["id"] == i ,["Point_1"]] = d[num[0]]
        df.loc[df["id"] == i ,["Point_2"]]  = d[num[2]]
        
        if num[1] > num[3]:
            d[num[0]] +=  3
        elif num[1] < num[3]:
            d[num[2]] +=  3
        else:
            d[num[0]] +=  1
            d[num[2]] +=  1
    
    return df,d
def preobr(df):
    if 'Unnamed: 0.1' in list(df.columns):
        df = df.drop('Unnamed: 0.1',axis=1)
    if 'Unnamed: 0' in list(df.columns):
        df = df.drop('Unnamed: 0',axis=1)
    
    df = df.drop('Матчи 1',axis=1)
    df = df.drop('Матчи 2',axis=1)
    df = df.astype({'счет1': 'int32'})
    df = df.astype({'счет2': 'int32'})

    df["тотал"] = df["счет1"] + df["счет2"]
    df["Дата"] = pd.to_datetime(df['Дата'],dayfirst=True)
    df['Температура'] = df['Температура'].apply(lambda x: int(x[1:-2]))
    
    for i in [14919463, 14898101]:
        df.loc[df["id"] == i ,['Офсайды  (соперник) 2']] = 4.00
        df.loc[df["id"] == i ,['Офсайды 2']] = 1.00
    
    #mask_1 = df["id"] == 14898101
    #df['Офсайды  (соперник) 2'][mask_1] = 4.00
    #df['Офсайды 2'][mask_1] = 1.00
    
    df["Point_1"] = 0
    df["Point_2"] = 0
    df["liga"] = df["общая информация"].apply(lambda x: x.split(",")[0])
    df["Tour"] = df["общая информация"].apply(lambda x: x.split(",")[1])
    
    df = df[df["Tour"] != " стыковые матчи"]
    df = df[df["Tour"] != " понижение/повышение - финал"]
    df = df[df["Tour"] != " дополнительный матч"]
    df["Tour"] = df["Tour"].apply(lambda x: int(x.strip()[:-6]))
    
    df = df.astype({'Офсайды  (соперник) 2': 'float32'})
    df = df.astype({'Офсайды 2': 'float32'})

    df["Сухие матчи 1"] = df["Сухие матчи 1"].apply(lambda x: int(x.split()[-1]))
    df["Сухие матчи 2"] = df["Сухие матчи 2"].apply(lambda x: int(x.split()[0]))

    df["обе забьют 1"] = df["обе забьют 1"].apply(lambda x: int(x.split()[-1]))
    df["обе забьют 2"] = df["обе забьют 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал больше 2.5 1"] = df["Тотал больше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал больше 2.5 2"] = df["Тотал больше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Тотал меньше 2.5 1"] = df["Тотал меньше 2.5 1"].apply(lambda x: int(x.split()[-1]))
    df["Тотал меньше 2.5 2"] = df["Тотал меньше 2.5 2"].apply(lambda x: int(x.split()[0]))

    df["Поражения 1"] = df["Поражения 1"].apply(lambda x: int(x.split()[-1]))
    df["Поражения 2"] = df["Поражения 2"].apply(lambda x: int(x.split()[0]))

    df["Ничьи 1"] = df["Ничьи 1"].apply(lambda x: int(x.split()[-1]))
    df["Ничьи 2"] = df["Ничьи 2"].apply(lambda x: int(x.split()[0]))

    df["Победы 1"] = df["Победы 1"].apply(lambda x: int(x.split()[-1]))
    df["Победы 2"] = df["Победы 2"].apply(lambda x: int(x.split()[0]))

    df["Отдых: дни (часы) 1"] = df["Отдых: дни (часы) 1"].apply(lambda x: float(x.split()[-1]))
    df["Отдых: дни (часы) 2"] = df["Отдых: дни (часы) 2"].apply(lambda x: float(x.split()[0]))
    
    #df = point_bund(df)
    return df
Bundes_liga_19_20_cl = preobr(Bundes_liga_19_20)
df, d = point_bund(Bundes_liga_19_20_cl)

In [52]:
d = pd.DataFrame.from_dict(d, orient='index')
d.columns = ["point"]
d.sort_values(by="point",ascending=False)

,point
Бавария,82
Боруссия Дортмунд,69
РБ Лейпциг,66
Боруссия М,65
Байер,63
Хоффенхайм,52
Вольфсбург,49
Фрайбург,48
Айнтрахт Франкфурт,45
Герта,41


In [ ]:
df = preobr(Bundes_liga_19_20)
df["rang_1"] = 0
df["rang_2"] = 0


d = pd.DataFrame({
    "team": list(df["Команда1"].value_counts().index),
    "point": [0] * len(list(df["Команда1"].value_counts().index)),
    "rang": [0] * len(list(df["Команда1"].value_counts().index))
})

df = df.sort_values(by="Tour",axis=0)

for i in list(df["id"]):
    num = list(df.loc[df["id"] == i,["Команда1", "счет1", "Команда2", "счет2"]].values[0])
    
    df.loc[df["id"] == i ,["Point_1"]] = d[num[0]]
    df.loc[df["id"] == i ,["Point_2"]]  = d[num[2]]
    
    if num[1] > num[3]:
        d[num[0]] +=  3
    elif num[1] < num[3]:
        d[num[2]] +=  3
    else:
        d[num[0]] +=  1
        d[num[2]] +=  1

In [63]:

d = pd.DataFrame({
    "team": list(df["Команда1"].value_counts().index),
    "point": [0] * len(list(df["Команда1"].value_counts().index)),
    "rang": [0] * len(list(df["Команда1"].value_counts().index))
})

d.loc[:,"rang"] = range(1,19)
display(d)
d.loc[:,"point"] = [0,1,3,0,0,1,1,3,3,1,3,0,1,0,3,0,1,3]
display(d)
d = d.sort_values(by="point",ascending=False)
display(d)
d.loc[:,"rang"] = range(1,19)
display(d)

,team,point,rang
0,Бавария,0,1
1,Унион Берлин,0,2
2,РБ Лейпциг,0,3
3,Шальке,0,4
4,Хоффенхайм,0,5
5,Фортуна Дюссельдорф,0,6
6,Майнц,0,7
7,Аугсбург,0,8
8,Падерборн 07,0,9
9,Кёльн,0,10


,team,point,rang
0,Бавария,0,1
1,Унион Берлин,1,2
2,РБ Лейпциг,3,3
3,Шальке,0,4
4,Хоффенхайм,0,5
5,Фортуна Дюссельдорф,1,6
6,Майнц,1,7
7,Аугсбург,3,8
8,Падерборн 07,3,9
9,Кёльн,1,10


,team,point,rang
17,Герта,3,18
2,РБ Лейпциг,3,3
14,Фрайбург,3,15
10,Боруссия Дортмунд,3,11
7,Аугсбург,3,8
8,Падерборн 07,3,9
1,Унион Берлин,1,2
16,Айнтрахт Франкфурт,1,17
12,Вольфсбург,1,13
9,Кёльн,1,10


,team,point,rang
17,Герта,3,1
2,РБ Лейпциг,3,2
14,Фрайбург,3,3
10,Боруссия Дортмунд,3,4
7,Аугсбург,3,5
8,Падерборн 07,3,6
1,Унион Берлин,1,7
16,Айнтрахт Франкфурт,1,8
12,Вольфсбург,1,9
9,Кёльн,1,10


In [59]:
len([0,1,3,0,0,1,1,3,3,1,3,0,1,0,3])

15